In [134]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [135]:
import pandas as pd
import re
import json
import h2o


from functools import lru_cache

from utils import get_data, Config

from tree import make_tree

In [136]:
def clean_url(url):
    return re.sub(r"browser_id=(\d+)", "browser_id=1", url)

def make_clickable(url):
    # Clickable links for debugging; set browser_id=2 (unknown) to not count them in our real data collection
    url = clean_url(url)
    return f'<a href="{url}" target="_blank">{url}</a>'

In [137]:
@lru_cache(maxsize=None)
def id_to_browser(id):
    return " ".join(df.loc[df["browser_id"] == id].iloc[0][["name", "os", "version", "automation_mode", "headless_mode"]].to_list())

## Load Data + Preprocessing

In [138]:
# Load all data
initial_data = """
SELECT "Result".*, 
"Response".raw_header, "Response".status_code, "Response".label, "Response".resp_type,
"Browser".name, "Browser".version, "Browser".headless_mode, "Browser".os, "Browser".automation_mode, "Browser".add_info
FROM "Result"
JOIN "Response" ON "Result".response_id = "Response".id JOIN "Browser" ON "Result".browser_id = "Browser".id
WHERE "Browser".name != 'Unknown';
"""
df = get_data(Config(), initial_data)

Connecting to the PostgreSQL database...
Connection successful


In [139]:
# Extra columns
df["outcome_str"] = df["outcome_value"].fillna("None").astype(str)

# Unify outcomes that are the same
# Firefox: {'error': 'object "TypeError: NetworkError when attempting to fetch resource."', 'headers': ''} Chromium: {'error': 'object "TypeError: Failed to fetch"', 'headers': ''}
# Safari: {'error': 'object "TypeError: Load failed"', 'headers': ''}
df["outcome_str"] = df["outcome_str"].replace("{'error': 'object \"TypeError: NetworkError when attempting to fetch resource.\"', 'headers': ''}", "{'error': 'object \"TypeError: Failed to fetch\"', 'headers': ''}")
df["outcome_str"] = df["outcome_str"].replace("{'error': 'object \"TypeError: Load failed\"', 'headers': ''}", "{'error': 'object \"TypeError: Failed to fetch\"', 'headers': ''}")

# For document referrer we do not want to know the exact resp_id and count
# Maybe we even only want to know whether it is a origin or the full URl?
df['outcome_str'] = df['outcome_str'].replace(r'resp_id=\d+', 'resp_id=<resp_id>', regex=True)
df['outcome_str'] = df['outcome_str'].replace(r'count=\d+', 'count=<count>', regex=True)

df["clean_url"] = df["full_url"].apply(clean_url)

df["browser"] = df["browser_id"].apply(id_to_browser)
df["org_origin"] = df["org_scheme"] + "://" + df["org_host"]
df["resp_origin"] = df["resp_scheme"] + "://" + df["resp_host"]

In [140]:
df["outcome_str"].unique()

array(["{'error': 'null', 'headers': 'access-control-allow-origin,content-length,date,server,test,'}",
       "{'error': 'null', 'headers': 'content-length,'}",
       '{\'error\': \'object "TypeError: Failed to fetch"\', \'headers\': \'\'}',
       "{'error': 'null', 'headers': 'access-control-allow-credentials,content-length,date,server,test,'}",
       "{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-origin,content-length,date,server,test,'}",
       'load', 'message timeout',
       "{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-origin,access-control-expose-headers,content-length,date,server,test,'}",
       "{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}",
       "{'error': 'null', 'headers': 'cache-control,content-type,'}",
      

## Overview

In [141]:
display(df.head(2))
display(df.describe())
display(df.columns)

,id,outcome_type,outcome_value,test_name,test_status,test_message,test_stack,org_scheme,org_host,resp_scheme,...,version,headless_mode,os,automation_mode,add_info,outcome_str,clean_url,browser,org_origin,resp_origin
0,192957,<class 'dict'>,"{'error': 'null', 'headers': 'access-control-a...",fetch_GET,0,None,None,http,sub.headers.websec.saarland,http,...,1.60.114 (119.0.6045.124),headless-new,Ubuntu 22.04,selenium,,"{'error': 'null', 'headers': 'access-control-a...",http://sub.headers.websec.saarland/_hp/tests/f...,brave Ubuntu 22.04 1.60.114 (119.0.6045.124) s...,http://sub.headers.websec.saarland,http://sub.headers.websec.saarland
1,192958,<class 'dict'>,"{'error': 'null', 'headers': 'access-control-a...",fetch_TEST,0,None,None,http,sub.headers.websec.saarland,http,...,1.60.114 (119.0.6045.124),headless-new,Ubuntu 22.04,selenium,,"{'error': 'null', 'headers': 'access-control-a...",http://sub.headers.websec.saarland/_hp/tests/f...,brave Ubuntu 22.04 1.60.114 (119.0.6045.124) s...,http://sub.headers.websec.saarland,http://sub.headers.websec.saarland


,id,test_status,browser_id,testcase_id,response_id,created_at,updated_at,status_code
count,209120.000000,209120.000000,209120.000000,209120.0,209120.000000,209120,209120,209120.000000
mean,297672.500000,0.000172,17.500000,1.0,79.027927,2023-12-20 10:47:21.098585856,2023-12-20 10:47:21.098587648,215.998470
min,192957.000000,0.000000,13.000000,1.0,4.000000,2023-12-20 09:58:40.566554,2023-12-20 09:58:40.566558,200.000000
25%,245236.750000,0.000000,15.000000,1.0,24.000000,2023-12-20 10:11:07.772600832,2023-12-20 10:11:07.772600832,200.000000
50%,297672.500000,0.000000,17.500000,1.0,47.000000,2023-12-20 10:51:28.313465088,2023-12-20 10:51:28.313468416,200.000000
75%,350108.250000,0.000000,20.000000,1.0,152.000000,2023-12-20 11:24:30.005777408,2023-12-20 11:24:30.005777664,200.000000
max,402388.000000,2.000000,22.000000,1.0,218.000000,2023-12-20 11:44:20.740761,2023-12-20 11:44:20.740763,302.000000
std,60503.039049,0.018555,2.872288,0.0,66.518621,NaN,NaN,37.093119


Index(['id', 'outcome_type', 'outcome_value', 'test_name', 'test_status',
       'test_message', 'test_stack', 'org_scheme', 'org_host', 'resp_scheme',
       'resp_host', 'relation_info', 'browser_id', 'testcase_id',
       'response_id', 'status', 'full_url', 'created_at', 'updated_at',
       'raw_header', 'status_code', 'label', 'resp_type', 'name', 'version',
       'headless_mode', 'os', 'automation_mode', 'add_info', 'outcome_str',
       'clean_url', 'browser', 'org_origin', 'resp_origin'],
      dtype='object')

In [142]:
# Collected results per browser
df.groupby(["browser_id", "name", "version", "os", "headless_mode", "automation_mode"])["id"].count().sort_values(ascending=False).to_frame()

,,,,,,id
browser_id,name,version,os,headless_mode,automation_mode,
13,chrome,119,Ubuntu 22.04,headless-new,selenium,20912
14,firefox,119,Ubuntu 22.04,headless,selenium,20912
15,edge,119,Ubuntu 22.04,headless-new,selenium,20912
16,brave,1.59.120 (118.0.5993.88),Ubuntu 22.04,headless-new,selenium,20912
17,brave,1.60.114 (119.0.6045.124),Ubuntu 22.04,headless-new,selenium,20912
18,chrome,119,Ubuntu 22.04,xvfb,selenium,20912
19,firefox,119,Ubuntu 22.04,xvfb,selenium,20912
20,edge,119,Ubuntu 22.04,xvfb,selenium,20912
21,brave,1.59.120 (118.0.5993.88),Ubuntu 22.04,xvfb,selenium,20912


In [145]:
# Timeouts
display(df.groupby(["browser_id", "name", "version", "os", "headless_mode", "automation_mode"])["test_status"].value_counts().to_frame())

display(df.loc[df["test_status"] == 2].head(3))

count
browser_id name    version                   os           headless_mode automation_mode test_status       
13         chrome  119                       Ubuntu 22.04 headless-new  selenium        0            20912
14         firefox 119                       Ubuntu 22.04 headless      selenium        0            20911
                                                                                        2                1
15         edge    119                       Ubuntu 22.04 headless-new  selenium        0            20910
                                                                                        2                2
16         brave   1.59.120 (118.0.5993.88)  Ubuntu 22.04 headless-new  selenium        0            20912
17         brave   1.60.114 (119.0.6045.124) Ubuntu 22.04 headless-new  selenium        0            20912
18         chrome  119                       Ubuntu 22.04 xvfb          selenium        0            20912
19         firefox 119                       Ubuntu 22.04 xvfb          selenium        0            20907
                                                                                        2                5
20         edge    119                       Ubuntu 22.04 xvfb          selenium        0            20902
                                                                                        2               10
21         brave   1.59.120 (118.0.5993.88)  Ubuntu 22.04 xvfb          selenium        0            20912
22         brave   1.60.114 (119.0.6045.124) Ubuntu 22.04 xvfb          selenium        0            20912

,id,outcome_type,outcome_value,test_name,test_status,test_message,test_stack,org_scheme,org_host,resp_scheme,...,version,headless_mode,os,automation_mode,add_info,outcome_str,clean_url,browser,org_origin,resp_origin
34765,227596,<class 'NoneType'>,None,oac_window.open,2,Test timed out,None,http,sub.headers.websec.saarland,https,...,119,headless-new,Ubuntu 22.04,selenium,,None,http://sub.headers.websec.saarland/_hp/tests/o...,edge Ubuntu 22.04 119 selenium headless-new,http://sub.headers.websec.saarland,https://headers.webappsec.eu
37784,230593,<class 'NoneType'>,None,oac_window.open,2,Test timed out,None,http,sub.headers.websec.saarland,http,...,119,xvfb,Ubuntu 22.04,selenium,,None,http://sub.headers.websec.saarland/_hp/tests/o...,edge Ubuntu 22.04 119 selenium xvfb,http://sub.headers.websec.saarland,http://headers.webappsec.eu
37785,230594,<class 'NoneType'>,None,oac_iframe,2,Test timed out,None,http,sub.headers.websec.saarland,https,...,119,xvfb,Ubuntu 22.04,selenium,,None,http://sub.headers.websec.saarland/_hp/tests/o...,edge Ubuntu 22.04 119 selenium xvfb,http://sub.headers.websec.saarland,https://headers.webappsec.eu


In [146]:
# Time taken!
# NOTE: this is now broken as we ran all tests on linux twice for a stability analysis
df.groupby(["name", "version", "os", "headless_mode", "automation_mode", "org_scheme"])["created_at"].agg(["min", "max"]).apply(lambda x: x["max"] - x["min"], axis=1).to_frame()

0
name    version                   os           headless_mode automation_mode org_scheme                       
brave   1.59.120 (118.0.5993.88)  Ubuntu 22.04 headless-new  selenium        http       0 days 01:29:15.190542
                                                                             https      0 days 01:26:48.371864
                                               xvfb          selenium        http       0 days 01:30:28.246903
                                                                             https      0 days 01:28:25.877146
        1.60.114 (119.0.6045.124) Ubuntu 22.04 headless-new  selenium        http       0 days 01:29:15.177088
                                                                             https      0 days 01:26:53.931604
                                               xvfb          selenium        http       0 days 01:30:25.268345
                                                                             https      0 days 01:28:07.876485
chrome  119                       Ubuntu 22.04 headless-new  selenium        http       0 days 01:29:06.393217
                                                                             https      0 days 01:26:43.887262
                                               xvfb          selenium        http       0 days 01:30:11.077591
                                                                             https      0 days 01:28:03.952745
edge    119                       Ubuntu 22.04 headless-new  selenium        http       0 days 01:23:53.035976
                                                                             https      0 days 01:27:01.534731
                                               xvfb          selenium        http       0 days 01:24:54.645264
                                                                             https      0 days 01:28:17.297944
firefox 119                       Ubuntu 22.04 headless      selenium        http       0 days 01:25:19.112014
                                                                             https      0 days 01:28:16.290569
                                               xvfb          selenium        http       0 days 01:25:46.942611
                                                                             https      0 days 01:27:57.987121

## Response analysis

- We have 10456 unique tests: (test_name x relation_info) x org_scheme x (resp_scheme x resp_host) x response_id)

In [147]:
# Unique tests
tests = {
    "Fetch": [4, 2, 8, 6], 
    "Framing": [9, 2, 8, 35], # 7+11+17
    "PP": [4, 2, 8, 24],
    "OAC": [3, 2, 8, 14],
    "TAO": [1, 2, 8, 13],
    "RP": [2, 2, 8, 16],
    "CSP-SCRIPT": [2, 2, 8, 17],
    "COEP": [2, 2, 8, 11],
    "CORP": [4, 2, 8, 11],
    "CSP-IMG": [1, 2, 8, 17],
    "HSTS": [2, 1, 2, 14],
    "COOP": [1, 2, 8, 11],
}
tests = pd.DataFrame.from_dict(tests, columns=["tests", "org_origins", "resp_origins", "responses"], orient="index")
tests["All tests"] = tests.prod(axis=1)
tests.loc["Sum"] = tests.sum(axis=0)
tests

,tests,org_origins,resp_origins,responses,All tests
Fetch,4,2,8,6,384
Framing,9,2,8,35,5040
PP,4,2,8,24,1536
OAC,3,2,8,14,672
TAO,1,2,8,13,208
RP,2,2,8,16,512
CSP-SCRIPT,2,2,8,17,544
COEP,2,2,8,11,352
CORP,4,2,8,11,704
CSP-IMG,1,2,8,17,272


In [148]:
# Only consider the tests that did not timeout! (we already reran them above)
df = df.loc[df["test_status"] == 0]

In [149]:
# SELECT count(*) as c2, count(DISTINCT browser_id), count(DISTINCT outcome_value) as c1, test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, response_id 
# FROM "Result" GROUP by test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, response_id order by c1 DESC, c2 DESC

res = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])[["browser_id", "outcome_str"]].agg(["count", "nunique"])
display(res)
display(res.value_counts().to_frame())

browser_id  \
                                                                                                                                    count   
test_name             org_scheme org_host                    resp_scheme resp_host                   relation_info response_id              
accesswindow_direct   http       sub.headers.websec.saarland http        headers.webappsec.eu        direct        71                  20   
                                                                                                                   72                  20   
                                                                                                                   73                  20   
                                                                                                                   74                  20   
                                                                                                                   75                  20   
...                                                                                                                                   ...   
upgradeHSTS_subdomain http       sub.headers.websec.saarland https       sub.headers.websec.saarland subdomain     139                 20   
                                                                                                                   140                 20   
                                                                                                                   141                 20   
                                                                                                                   142                 20   
                                                                                                                   143                 20   

                                                                                                                                        \
                                                                                                                               nunique   
test_name             org_scheme org_host                    resp_scheme resp_host                   relation_info response_id           
accesswindow_direct   http       sub.headers.websec.saarland http        headers.webappsec.eu        direct        71               10   
                                                                                                                   72               10   
                                                                                                                   73               10   
                                                                                                                   74               10   
                                                                                                                   75               10   
...                                                                                                                                ...   
upgradeHSTS_subdomain http       sub.headers.websec.saarland https       sub.headers.websec.saarland subdomain     139              10   
                                                                                                                   140              10   
                                                                                                                   141              10   
                                                                                                                   142              10   
                                                                                                                   143              10   

                                                                                                                               outcome_str  \
                                                                             

count
(browser_id, count) (browser_id, nunique) (outcome_str, count) (outcome_str, nunique)       
20                  10                    20                   1                        9354
                                                               2                         823
                                                               3                         263
19                  10                    19                   1                          14
18                  10                    18                   1                           1
                    9                     18                   1                           1

## Stability analysis
- Does the same browser/env always result in the same outcomes?
- We ran all tests on linux twice, does any of these tests have more than one result?
    - A total of 20/10456 tests have different outcomes
    - ~~2xreferrer_iframe (both timeout of the test "None" vs "message timeout" result); can be fixed by removing timeouts first~~
    - 18x subresourceloadingCOEP_img, all differences: 'swag-same-origin.jpg': 'error' | 'load'
        - trying to manually reproduce: e.g., http://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?resp_type=basic&browser_id=2&label=COEP&first_id=57&last_id=57&scheme=http
        - devtools open: with cache disabeled: 15x error, without cache disabled: 13x or 16x error?
        - devtools closed: 13x or 16x error (tried to fix with Cache-Control: no-cache, no-store but does not seem to have an effect)
        - chrome always 14x error
    - 13x firefox xvfb, 6x firefox headless, 1x edge

In [150]:
stab = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser_id"])[["outcome_str"]].agg(["count", "nunique"])
# Tests with more than one outcome
diff_outcomes = stab.loc[stab[("outcome_str", "nunique")] != 1]
print(f"{len(diff_outcomes)} tests have different outcomes!")
display(diff_outcomes.reset_index()["browser_id"].apply(id_to_browser).value_counts().to_frame())
display(diff_outcomes)

30 tests have different outcomes!


,count
browser_id,
firefox Ubuntu 22.04 119 selenium headless,18
firefox Ubuntu 22.04 119 selenium xvfb,12


outcome_str  \
                                                                                                                                                         count   
test_name                  org_scheme org_host                    resp_scheme resp_host                       relation_info response_id browser_id               
subresourceloadingCOEP_img http       sub.headers.websec.saarland http        headers.websec.saarland         direct        58          19                   2   
                                                                                                                            60          19                   2   
                                                                                                                            61          14                   2   
                                                                                                                                        19                   2   
                                                                                                                            62          14                   2   
                                                                                                                            63          14                   2   
                                                                                                                            65          14                   2   
                                                                                                                                        19                   2   
                                                                                                                            66          14                   2   
                                                                              sub.headers.websec.saarland     direct        58          19                   2   
                                                                                                                            60          19                   2   
                                                                                                                            61          14                   2   
                                                                                                                                        19                   2   
                                                                                                                            62          14                   2   
                                                                                                                            63          14                   2   
                                                                                                                            65          14                   2   
                                                                                                                                        19                   2   
                                                                                                                            66          14                   2   
                                                                              sub.sub.headers.websec.saarland direct        58          19                   2   
                                                                                                                            60          19                   2   
                                                                                                                            61          14                   2   
                                                                                                                                        19                   2   
                                                                                                 

In [152]:
for test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, response_id, browser_id in stab.loc[stab[("outcome_str", "nunique")] != 1].index:
    print(test_name)
    rows = df.loc[(df["org_scheme"] == org_scheme) & (df["test_name"] == test_name) & (df["resp_scheme"] == resp_scheme) & (df["resp_host"] == resp_host) & (df["relation_info"] == relation_info) & (df["response_id"] == response_id) & (df["browser_id"] == browser_id)]
    with pd.option_context("display.max_colwidth", 200):
        #display(rows[["outcome_str", "name", "headless_mode", "os", "version"]].value_counts().to_frame())
        display(rows[["outcome_str", "name"]].value_counts().to_frame())
        disp = rows.drop_duplicates(subset="outcome_str")[["browser_id", "headless_mode", "name", "outcome_str", "raw_header", "test_name", "org_scheme", "org_host", "resp_scheme", "relation_info", "resp_host", "full_url"]]
        disp = disp.style.format({'full_url': make_clickable})
        display(disp)
        

subresourceloadingCOEP_img


,,count
outcome_str,name,
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",firefox,1
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}",firefox,1


,browser_id,headless_mode,name,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
43675,19,xvfb,firefox,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","[['Cross-Origin-Embedder-Policy', 'unsafe-none']]",subresourceloadingCOEP_img,http,sub.headers.websec.saarland,http,direct,headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?resp_type=basic&browser_id=1&label=COEP&first_id=58&last_id=58&scheme=http
147666,19,xvfb,firefox,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","[['Cross-Origin-Embedder-Policy', 'unsafe-none']]",subresourceloadingCOEP_img,http,sub.headers.websec.saarland,http,direct,headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?timeout=5&resp_type=basic&browser_id=1&label=COEP&first_id=58&last_id=58&scheme=http


subresourceloadingCOEP_img


,,count
outcome_str,name,
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",firefox,1
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}",firefox,1


...

,,count
outcome_str,name,
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",firefox,1
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}",firefox,1


,browser_id,headless_mode,name,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
91899,14,headless,firefox,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","[['Cross-Origin-Embedder-Policy', 'unsafe-none']]",subresourceloadingCOEP_img,https,sub.headers.websec.saarland,https,direct,sub.headers.websec.saarland,https://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?resp_type=basic&browser_id=1&label=COEP&first_id=58&last_id=58&scheme=https
197323,14,headless,firefox,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","[['Cross-Origin-Embedder-Policy', 'unsafe-none']]",subresourceloadingCOEP_img,https,sub.headers.websec.saarland,https,direct,sub.headers.websec.saarland,https://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?timeout=5&resp_type=basic&browser_id=1&label=COEP&first_id=58&last_id=58&scheme=https


subresourceloadingCOEP_img


,,count
outcome_str,name,
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",firefox,1
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}",firefox,1


,browser_id,headless_mode,name,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
91907,14,headless,firefox,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","[['Cross-Origin-Embedder-Policy', 'unsafe-none']]",subresourceloadingCOEP_img,https,sub.headers.websec.saarland,https,direct,sub.sub.headers.websec.saarland,https://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?resp_type=basic&browser_id=1&label=COEP&first_id=58&last_id=58&scheme=https
197331,14,headless,firefox,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","[['Cross-Origin-Embedder-Policy', 'unsafe-none']]",subresourceloadingCOEP_img,https,sub.headers.websec.saarland,https,direct,sub.sub.headers.websec.saarland,https://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?timeout=5&resp_type=basic&browser_id=1&label=COEP&first_id=58&last_id=58&scheme=https


## Differences:
- COOP (accesswindow_direct):
    - [ ] Edge + Firefox: redirect first
    - [ ] Brave + Chrome: COOP first?
-  Framing:
    -  [ ] framing_iframe_sandbox: CSP-FA *; Firefox allows, all other browsers deny?
- PP not supported in Firefox
- OAC not supported in Firefox
    - [ ] Also random/caching behavior?
- PerfAPI:
    - [ ] Chrome/Firefox have different rules when to create an entry at all?
- COEP (CORP):
    - [ ] Firefox incorrect behavior for same-origin (applies same-site?)
- CORP:
    - [ ] same-origin: Edge does not check the scheme?
    - [x] Firefox applies CORP on object tags (even though it should not!)
    - [ ] Some other CORP and/or COEP strange stuff (Firefox)
- Referrer Policy: different default in Brave (unsafe-url is never allowed)
- Subresource-loading (CSP): differences due to HTTPS Upgrades
- Script Execution (CSP):
    - [ ] Safari sandbox bug?! (maybe related https://bugs.webkit.org/show_bug.cgi?id=223848 unsure though)
- Upgrade (HSTS):
    - [ ] Safari bug?

In [153]:
def get_uniques(df):
    browsers = f"{df['name'].unique().tolist()}"
    try:
        outcome = unique_outcomes[df.name[0]]
    except KeyError:
        outcome = {}
        #outcome = []
    outcome[browsers] = df.name[1]
    #outcome.append(f"{df.name[1]} ({browsers})")
    unique_outcomes[df.name[0]] = outcome

for grouping, group in df.groupby(["test_name", "relation_info"]):
    
    unique_outcomes = {}
    group.groupby(["name"])["outcome_str"].value_counts(normalize=True).reset_index().groupby(["outcome_str", "proportion"], group_keys=True).apply(get_uniques)
    d = pd.DataFrame.from_dict(unique_outcomes, orient="index")
    if d.shape[1] != 1:
        print(grouping)
        # display(unique_outcomes)
        display(group.groupby(["name"])["outcome_str"].value_counts(normalize=True).to_frame().unstack(0))
        for value, dic in unique_outcomes.items():
            values = [f"{v} ({k})" for k,v in dic.items()]
            # print(f"{value}: {values}")
        display(d)
        print()


('accesswindow_direct', 'direct')


proportion            \
name                                                    brave    chrome   
outcome_str                                                               
{'window.open.opener': 'null'}                       0.340909  0.340909   
{'window.open.opener': 'object "[object Window]"'}   0.659091  0.659091   

                                                                        
name                                                    edge   firefox  
outcome_str                                                             
{'window.open.opener': 'null'}                      0.181818  0.181818  
{'window.open.opener': 'object "[object Window]"'}  0.818182  0.818182

,"['edge', 'firefox']","['brave', 'chrome']"
{'window.open.opener': 'null'},0.181818,0.340909
"{'window.open.opener': 'object ""[object Window]""'}",0.818182,0.659091



('framing_iframe', 'sandbox')


proportion                              
name                 brave    chrome      edge   firefox
outcome_str                                             
message send      0.157143  0.157143  0.157143  0.171429
message timeout   0.842857  0.842857  0.842857  0.828571

...

proportion                              
name             brave    chrome      edge   firefox
outcome_str                                         
error         0.295455  0.295455  0.295455  0.301136
load          0.704545  0.704545  0.704545  0.698864

,"['brave', 'chrome', 'edge']",['firefox']
error,0.295455,0.301136
load,0.704545,0.698864



('subresourceloadingCORP_object', 'direct')


proportion                       
name             brave chrome  edge   firefox
outcome_str                                  
error             0.25   0.25  0.25  0.431818
load              0.75   0.75  0.75  0.568182

,"['brave', 'chrome', 'edge']",['firefox']
error,0.25,0.431818
load,0.75,0.568182


In [154]:
test_name_seen = set()
for test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, response_id in res.loc[res[("outcome_str", "nunique")] != 1].index:
    # Only look at each feature group once
    if test_name in test_name_seen:
        continue
    test_name_seen.add(test_name)
    print(test_name)
    rows = df.loc[(df["org_scheme"] == org_scheme) & (df["test_name"] == test_name) & (df["resp_scheme"] == resp_scheme) & (df["resp_host"] == resp_host) & (df["relation_info"] == relation_info) & (df["response_id"] == response_id)]
    with pd.option_context("display.max_colwidth", 200):
        #display(rows[["outcome_str", "name", "headless_mode", "os", "version"]].value_counts().to_frame())
        display(rows[["outcome_str", "name"]].value_counts().to_frame())
        disp = rows.drop_duplicates(subset="outcome_str")[["browser_id", "headless_mode", "name", "outcome_str", "raw_header", "test_name", "org_scheme", "org_host", "resp_scheme", "relation_info", "resp_host", "full_url"]]
        disp = disp.style.format({'full_url': make_clickable})
        display(disp)
        
    

accesswindow_direct


count
outcome_str                                        name          
{'window.open.opener': 'null'}                     brave        8
                                                   chrome       4
{'window.open.opener': 'object "[object Window]"'} edge         4
                                                   firefox      4

,browser_id,headless_mode,name,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
44473,15,headless-new,edge,"{'window.open.opener': 'object ""[object Window]""'}","[['Cross-Origin-Opener-Policy', 'same-origin-allow-popups']]",accesswindow_direct,http,sub.headers.websec.saarland,http,direct,headers.webappsec.eu,http://sub.headers.websec.saarland/_hp/tests/window-references-coop.sub.html?resp_type=basic&browser_id=1&label=COOP&first_id=72&last_id=72&scheme=http
57907,13,headless-new,chrome,{'window.open.opener': 'null'},"[['Cross-Origin-Opener-Policy', 'same-origin-allow-popups']]",accesswindow_direct,http,sub.headers.websec.saarland,http,direct,headers.webappsec.eu,http://sub.headers.websec.saarland/_hp/tests/window-references-coop.sub.html?resp_type=basic&browser_id=1&label=COOP&first_id=72&last_id=72&scheme=http


framing_iframe


count
outcome_str     name          
message timeout brave        8
message send    firefox      4
message timeout chrome       4
                edge         4

...

count
outcome_str name          
load        brave        8
error       firefox      4
load        chrome       4
            edge         4

,browser_id,headless_mode,name,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
89130,15,headless-new,edge,load,"[['Cross-Origin-Resource-Policy', 'same-origin']]",subresourceloadingCORP_img,https,sub.headers.websec.saarland,http,direct,sub.headers.websec.saarland,https://sub.headers.websec.saarland/_hp/tests/subresource-loading-corp.sub.html?resp_type=basic&browser_id=1&label=CORP&first_id=46&last_id=46&scheme=https
94460,14,headless,firefox,error,"[['Cross-Origin-Resource-Policy', 'same-origin']]",subresourceloadingCORP_img,https,sub.headers.websec.saarland,http,direct,sub.headers.websec.saarland,https://sub.headers.websec.saarland/_hp/tests/subresource-loading-corp.sub.html?resp_type=basic&browser_id=1&label=CORP&first_id=46&last_id=46&scheme=https


subresourceloadingCORP_object


count
outcome_str name          
load        brave        8
error       firefox      4
load        chrome       4
            edge         4

,browser_id,headless_mode,name,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
41716,15,headless-new,edge,load,"[['Cross-Origin-Resource-Policy', 'same-origin']]",subresourceloadingCORP_object,http,sub.headers.websec.saarland,http,direct,headers.webappsec.eu,http://sub.headers.websec.saarland/_hp/tests/subresource-loading-corp.sub.html?resp_type=basic&browser_id=1&label=CORP&first_id=46&last_id=46&scheme=http
43727,14,headless,firefox,error,"[['Cross-Origin-Resource-Policy', 'same-origin']]",subresourceloadingCORP_object,http,sub.headers.websec.saarland,http,direct,headers.webappsec.eu,http://sub.headers.websec.saarland/_hp/tests/subresource-loading-corp.sub.html?resp_type=basic&browser_id=1&label=CORP&first_id=46&last_id=46&scheme=http


## Clustering and co.
- Maybe try decision trees again?
- One tree for each feature group?/label?/test_name?
    - outcome = outcome_str (mostly binary, sometimes more options)
    - variables:
        - browser (includes os, automation mode, headless mode, version)
        - test_id = test_name + relation_info (or only relation_info if one tree per test_name is created)
        - org_orgin = org_scheme + org_host
        - resp_origin = resp_scheme + resp_host
        - response_id or raw_headers + status_code
        - ...
     
- H2O setup:
    - `sudo apt install openjdk-17-jre-headless graphviz`
    - `wget https://h2o-release.s3.amazonaws.com/h2o/rel-3.44.0/2/h2o-3.44.0.2.zip`
    - `unzip h2o-3.44.0.2.zip`
    - `cd h2o-3.44.0.2` and `java -jar h2o.jar`

In [155]:
df[["test_name", "label", "relation_info"]].value_counts()

test_name                      label       relation_info 
fullscreen_iframe              PP          child_allow       7680
                                           child             7680
                                           direct            7680
                                           child_sandbox     7680
framing_iframe                 CSP-FA      nested            5440
                                           sandbox           5440
framing_object                 CSP-FA      sandbox           5440
                                           direct            5440
                                           nested            5440
framing_iframe                 CSP-FA      direct            5440
framing_embed                  CSP-FA      sandbox           5440
script_execution_iframe        CSP-SCRIPT  direct            5440
                                           sandbox           5440
imgloading_iframe              CSP-IMG     direct            5440
framing_embed     

In [164]:
h2o.init(nthreads=50, max_mem_size="100G", log_level="WARN")
h2o.no_progress()


# Only keep one row for each test that had non-deterministic results (this should only be Firefox COEP same-origin 18x)
print(len(df))
tree_df = df.drop_duplicates(subset=["test_name", "relation_info", "browser", "org_origin", "resp_origin", "response_id"])
print(len(tree_df))

# Idea limit on responses that have more than one outcome!
# This will remove all the branches that are the same in all browsers (and make trees large and confusing)
condition = tree_df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])["outcome_str"].transform("nunique") != 1
tree_df = tree_df.loc[condition]
print(len(tree_df))

for group_name, group in tree_df.groupby(["test_name", "relation_info"]):
    print(group_name)
    # pred_cols = ["browser", "relation_info", "org_origin", "resp_origin", "status_code", "raw_header"]
    pred_cols = ["browser", "org_origin", "resp_origin", "status_code", "raw_header"]

    group = group[["outcome_str", *pred_cols]]
    # Convert all columns to str, then to category
    tree = make_tree(group, pred_cols, group_name)
    # print(tree)


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,13 mins 27 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,1 month and 13 days
H2O_cluster_name:,H2O_from_python_ubuntu_uy37br
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.9 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


209102
104559
10820
('accesswindow_direct', 'direct')
Connecting to H2O server at http://localhost:54321 ... successful.


H2O_cluster_uptime:,13 mins 28 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,1 month and 13 days
H2O_cluster_name:,H2O_from_python_ubuntu_uy37br
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.9 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('accesswindow_direct', 'direct'), datapoints: 280
('framing_iframe', 'sandbox')
Connecting to H2O server at http://localhost:54321 ... successful.


...

Create tree: ('subresourceloadingCOEP_img', 'direct'), datapoints: 490
('subresourceloadingCORP_img', 'direct')
Connecting to H2O server at http://localhost:54321 ... successful.


H2O_cluster_uptime:,13 mins 42 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,1 month and 13 days
H2O_cluster_name:,H2O_from_python_ubuntu_uy37br
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.9 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('subresourceloadingCORP_img', 'direct'), datapoints: 10
('subresourceloadingCORP_object', 'direct')
Connecting to H2O server at http://localhost:54321 ... successful.


H2O_cluster_uptime:,13 mins 43 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,1 month and 13 days
H2O_cluster_name:,H2O_from_python_ubuntu_uy37br
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.9 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('subresourceloadingCORP_object', 'direct'), datapoints: 320


# Crash analysis OLD

### Exceptions thrown
- Fixed:
    - Only Firefox; both headless + Xvfb
        - OAC + RP
        - Both on HTTP and HTTPS
        - Response_ids: 185, 186, 341, 342; these are the ones that have status code 302!
        - Reason seems to be that Firefox switches to another tab in these cases and thus the wait for crashes. Solution: forcefully switch back to the original tab before waiting!
    - Only Brave: both headless + Xvfb
        - OAC + RP
        - Both on HTTP and HTTPS
        - Reason seems to be the tab switching, if deactivated no issue occurs

In [165]:
with open("/home/ubuntu/wpt/_hp/tools/crawler/desktop-selenium-2023-11-20 16:09:24.649008.log", "r") as f:
    crashes = []
    for l in f.readlines():
        if "649008: http" in l:
            match = re.match(".*(http|https)://.*browser_id=(\d+)&label=(.*)&first_id=(\d+).*", l)
            crashes.append([match[1], match[2], match[3], match[4]])

In [166]:
crashes = pd.DataFrame(crashes, columns=["scheme", "browser_id", "label", "response_id"]).drop_duplicates()
crashes

,scheme,browser_id,label,response_id
0,http,59,RP,172
2,http,59,RP,173
4,https,59,OAC,339
6,https,59,RP,181
8,http,75,RP,171
10,http,75,RP,172
12,https,75,OAC,330
14,https,75,OAC,338
16,https,75,RP,180
18,https,74,OAC,337
